In [ ]:
%pylab inline
import gc
import os
import sys
import glob
import json
import shutil
import shapely
import itertools
import importlib
import pandas as pd
import xarray as xr
from numpy import pi
import geopandas as gp
import summa_plot as sp
import dask.array as da
import matplotlib as mpl
from pprint import pprint 
from scipy import ndimage
from scipy.linalg import det
from functools import reduce
from jupyterthemes import jtplot
import matplotlib.colors as colors
from scipy.special import gamma,psi
from rasterio import features
from affine import Affine
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from sklearn.neighbors import NearestNeighbors
from summa_plot.spatial import add_map_features
jtplot.style(jtplot.infer_theme(), fscale=3.0)
jtplot.figsize(x=30, y=22)

In [ ]:
SHAPEFILE= './data/shapefile.shp'
WILLAMETTE = './data/subshapes/willamette2.shp'
SNAKE = './data/subshapes/snake2.shp'
ROCKIES = './data/subshapes/can_rockies.shp'
OLYMPIC = './data/subshapes/olympics.shp'
NCASCADES = './data/subshapes/north_cascades.shp'
gdf_all = gp.GeoDataFrame.from_file(SHAPEFILE)
gdf_all = gdf_all.to_crs({'init': 'epsg:4326'})
gdf_will = gp.GeoDataFrame.from_file(WILLAMETTE)
gdf_will = gdf_will.to_crs({'init': 'epsg:4326'})
gdf_snake = gp.GeoDataFrame.from_file(SNAKE)
gdf_snake = gdf_snake.to_crs({'init': 'epsg:4326'})
gdf_rockies = gp.GeoDataFrame.from_file(ROCKIES)
gdf_rockies = gdf_rockies.to_crs({'init': 'epsg:4326'})
gdf_olys = gp.GeoDataFrame.from_file(OLYMPIC)
gdf_olys = gdf_olys.to_crs({'init': 'epsg:4326'})
gdf_cascade = gp.GeoDataFrame.from_file(NCASCADES)
gdf_cascade = gdf_cascade.to_crs({'init': 'epsg:4326'})

ds = xr.open_dataset('../SUMMA_CRB_Setup/summa_run/summa_zLocalAttributes_columbia_gru.nc')
ds = xr.open_dataset('../SUMMA_CRB_Setup/summa_run/summa_zLocalAttributes_fix_elevation.nc')
gdf_all['elevation'] = ds['elevation']

In [ ]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

cmap = plt.get_cmap('gist_earth')
new_cmap = truncate_colormap(cmap, 0.3, 1.0) 

In [ ]:
ax = plt.axes(projection=ccrs.Mercator())
add_map_features(ax, states_provinces=False)
gdf_all.plot(ax=ax, column='elev', cmap=new_cmap, 
             transform=ccrs.PlateCarree(), linewidth=60, vmin=0)

color = 'black'
alpha = 0.2
gdf_rockies.plot(ax=ax, color=color, edgecolor=color, 
                 alpha=alpha, transform=ccrs.PlateCarree())
gdf_olys.plot(ax=ax, color=color, edgecolor=color, 
              alpha=alpha, transform=ccrs.PlateCarree())
gdf_will.plot(ax=ax, color=color, edgecolor=color,    
              alpha=alpha, transform=ccrs.PlateCarree())
gdf_snake.plot(ax=ax, color=color, edgecolor=color,   
               alpha=alpha, transform=ccrs.PlateCarree())

transform = ccrs.Mercator()
x, y = transform.transform_point(-122.75, 43.9, ccrs.PlateCarree())
plt.text(x, y, 'Willamette')

x, y = transform.transform_point(-115.5, 43.75, ccrs.PlateCarree())
plt.text(x, y, 'Snake')

x, y = transform.transform_point(-123, 46.6, ccrs.PlateCarree())
plt.text(x, y, 'Olympic \n Mountains')

x, y = transform.transform_point(-116.75, 51.2, ccrs.PlateCarree())
plt.text(x, y, 'Rocky \n Mountains')

x, y = transform.transform_point(-105, 49.2, ccrs.PlateCarree())
plt.text(x, y, 'Elevation (m)', rotation=90, fontsize=42)